# Análisis de Ventas - Demo Técnica

Este notebook demuestra el uso de los principales componentes del sistema:
- Conexión a la base de datos con SQLAlchemy
- Resultados de queries convertidos a pandas DataFrame
- Patrón Singleton aplicado a la clase de conexión
- Ejecución de pruebas unitarias con `pytest`

### Configurar la importacion correcta de modulos

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

### Conexion a la base de datos

In [2]:
from src.database.connection import DatabaseConnection
db = DatabaseConnection()
session = db.get_session()
db.close_session(session)
print("Conexión y sesión establecida correctamente.")

2025-06-04 19:02:36 - INFO - src.database.connection - Obteniendo sesión de la base de datos...
2025-06-04 19:02:36 - INFO - src.database.connection - Cerrando sesión de la base de datos...


Conexión y sesión establecida correctamente.


### Ejecucion de queries

In [3]:
# Ejecución de query y conversión a DataFrame
df = db.execute_query_as_dataframe("SELECT * FROM categories")
df.head()

2025-06-04 19:02:38 - INFO - src.database.connection - Ejecutando consulta: SELECT * FROM categories


,CategoryID,CategoryName
0,1,Confections
1,2,Shell fish
2,3,Cereals
3,4,Dairy
4,5,Beverages


In [4]:
df_customers = db.execute_query_as_dataframe("""
    SELECT c.CustomerID, c.FirstName, c.LastName, ct.CityName
    FROM customers c
    JOIN cities ct ON c.CityID = ct.CityID
""")
df_customers.head()

2025-06-04 19:02:41 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT c.CustomerID, c.FirstName, c.LastName, ct.CityName
    FROM customers c
    JOIN cities ct ON c.CityID = ct.CityID



,CustomerID,FirstName,LastName,CityName
0,1,Stefanie,Frye,Oklahoma
1,2,Sandy,Kirby,Pittsburgh
2,3,Lee,Zhang,Houston
3,4,Regina,Avery,Cleveland
4,5,Daniel,Mccann,Buffalo


In [21]:
df_products = db.execute_query_as_dataframe("""
    SELECT p.ProductID, p.ProductName, p.Price, cat.CategoryName
    FROM products p
    JOIN categories cat ON p.CategoryID = cat.CategoryID
""")
df_products.head()

2025-06-04 18:54:08 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT p.ProductID, p.ProductName, p.Price, cat.CategoryName
    FROM products p
    JOIN categories cat ON p.CategoryID = cat.CategoryID



,ProductID,ProductName,Price,CategoryName
0,1,Flour - Whole Wheat,74.2988,Cereals
1,2,Cookie Chocolate Chip With,91.2329,Cereals
2,3,Onions - Cippolini,9.1379,Poultry
3,4,Sauce - Gravy; Au Jus; Mix,54.3055,Poultry
4,5,Artichokes - Jerusalem,65.4771,Shell fish


In [5]:
df_sales = db.execute_query_as_dataframe("""
    SELECT SalesID, SalesDate, Quantity, TotalPrice, CustomerID
    FROM sales
    ORDER BY SalesDate DESC
    LIMIT 5
""")
df_sales.head()

2025-06-04 19:02:44 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT SalesID, SalesDate, Quantity, TotalPrice, CustomerID
    FROM sales
    ORDER BY SalesDate DESC
    LIMIT 5



,SalesID,SalesDate,Quantity,TotalPrice,CustomerID
0,417682,2 days 11:59:01,5,65.00,18075
1,3692623,2 days 11:59:01,3,51.00,8986
2,117578,2 days 11:59:00,5,45.00,17746
3,1944938,2 days 11:59:00,24,144.00,94027
4,2274474,2 days 11:59:00,16,80.00,61064


In [6]:
df_employees = db.execute_query_as_dataframe("""
    SELECT e.EmployeeID, e.FirstName, e.LastName, e.Gender, ct.CityName
    FROM employees e
    JOIN cities ct ON e.CityID = ct.CityID
""")
df_employees.head()

2025-06-04 19:02:46 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT e.EmployeeID, e.FirstName, e.LastName, e.Gender, ct.CityName
    FROM employees e
    JOIN cities ct ON e.CityID = ct.CityID



,EmployeeID,FirstName,LastName,Gender,CityName
0,1,Nicole,Fuller,F,New Orleans
1,2,Christine,Palmer,F,Fremont
2,3,Pablo,Cline,M,Rochester
3,4,Darnell,Nielsen,M,Lubbock
4,5,Desiree,Stuart,F,Anaheim


### Patrón de diseño aplicado: Singleton y Ejecucion de test
La clase `DatabaseConnection` implementa el patrón Singleton, garantizando una única instancia global:
```python
db1 = DatabaseConnection()
db2 = DatabaseConnection()
assert db1 is db2
```

In [7]:
#Ejecución de pruebas unitarias incluyendo el test unitario de una unica conexion a la base de datos
!pytest ../test -v

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.0.2, pluggy-1.5.0 -- C:\Users\bcami\OneDrive\Escritorio\ACCENTURE\sales_company\venv_sales_company\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\bcami\OneDrive\Escritorio\ACCENTURE\sales_company
plugins: anyio-4.8.0, cov-6.0.0
collecting ... collected 12 items

..\test\test_customer.py::test_get_full_name PASSED                      [  8%]
..\test\test_customer.py::test_set_address PASSED                        [ 16%]
..\test\test_customer.py::test_customer_repr PASSED                      [ 25%]
..\test\test_database_connection.py::test_singleton_instance PASSED      [ 33%]
..\test\test_database_connection.py::test_get_session PASSED             [ 41%]
..\test\test_database_connection.py::test_close_session PASSED           [ 50%]
..\test\test_database_connection.py::test_execute_query_as_dataframe PASSED [ 58%]
..\test\test_employee.py::test_employe